##### 1. Import Libraries

In [1]:
import pandas as pd
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
import plotly.colors

##### 2. Load and Explore the Data

In [2]:
df = pd.read_csv('online_retail.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


##### 3. Data Cleaning <br>

Remove rows with missing `CustomerID` values to ensure accurate segmentation.

In [5]:
df.dropna(subset=['CustomerID'], inplace=True)

##### 4. Feature Engineering <br>

Convert the `InvoiceDate` column to datetime format and calculate the total transaction amount.

In [6]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['TotalAmount'] = df['Quantity'] * df['UnitPrice']

In [7]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalAmount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


##### 5. RFM Reference Date <br>

Set the reference date for RFM analysis (usually the day after the last transaction).

In [8]:
reference_date = df['InvoiceDate'].max() + dt.timedelta(days=1)
reference_date

Timestamp('2011-12-10 12:50:00')

##### 6. Calculate RFM Metrics <br>

Aggregate the data to compute Recency, Frequency, and Monetary values for each customer.

In [9]:
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days, # Recency
    'InvoiceNo': 'count', # Frequency
    'TotalAmount': 'sum' # Monetary
})

In [10]:
rfm.rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalAmount': 'Monetary'
}, inplace=True)

In [11]:
rfm.head()

,Recency,Frequency,Monetary
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


##### 7. Assign RFM Scores <br>

Score each customer on Recency, Frequency, and Monetary metrics using quantiles.

In [12]:
quantiles = rfm.quantile(q=[0.25, 0.5, 0.75])

def RScore(x, p, d):
    # x: value to score
    # p: column name (Recency, Frequency, Monetary)
    # d: dataframe with quantiles

    # Lower recency has better scores
    if p == 'Recency': 
        if x <= d[p][0.25]:
            return 4
        elif x <= d[p][0.5]:
            return 3
        elif x <= d[p][0.75]:
            return 2
        else:
            return 1
    # Higher frequency and monetary has better scores
    else: 
        if x <= d[p][0.25]:
            return 1
        elif x <= d[p][0.5]:
            return 2
        elif x <= d[p][0.75]:
            return 3
        else:
            return 4

In [13]:
quantiles

,Recency,Frequency,Monetary
0.25,17.0,17.0,293.3625
0.50,50.0,42.0,648.0750
0.75,143.0,102.0,1611.7250


In [14]:
# Apply RScore function to each column
rfm['R'] = rfm['Recency'].apply(RScore, args=('Recency', quantiles))
rfm['F'] = rfm['Frequency'].apply(RScore, args=('Frequency', quantiles))
rfm['M'] = rfm['Monetary'].apply(RScore, args=('Monetary', quantiles))

In [15]:
rfm['RFM_Score'] = rfm[['R', 'F', 'M']].sum(axis=1)
rfm.head()

,Recency,Frequency,Monetary,R,F,M,RFM_Score
CustomerID,,,,,,,
12346.0,326,2,0.00,1,1,1,3
12347.0,2,182,4310.00,4,4,4,12
12348.0,75,31,1797.24,2,2,4,8
12349.0,19,73,1757.55,3,3,4,10
12350.0,310,17,334.40,1,1,2,4


##### 8. RFM Segmentation

In [16]:
def assign_segment(score):
    if score < 5:
        return 'Low-Value'  
    elif score < 9:
        return 'Mid-Value' 
    else:
        return 'High-Value' 

rfm['RFM_Segment'] = rfm['RFM_Score'].apply(assign_segment)

In [17]:
segment_counts = rfm['RFM_Segment'].value_counts().reset_index()
segment_counts.columns = ['RFM_Segment', 'Count']
segment_order = ['Low-Value', 'Mid-Value', 'High-Value']
segment_counts['RFM_Segment'] = pd.Categorical(segment_counts['RFM_Segment'], categories=segment_order, ordered=True)
segment_counts = segment_counts.sort_values('RFM_Segment')
segment_counts


,RFM_Segment,Count
2,Low-Value,783
0,Mid-Value,1899
1,High-Value,1690


In [18]:
pastel_colors = px.colors.qualitative.Pastel
color_map = {
    'High-Value': pastel_colors[0],  
    'Low-Value': pastel_colors[2],   
    'Mid-Value': pastel_colors[1]    
}

fig = px.bar(segment_counts,
             x='RFM_Segment',
             y='Count',
             title='Customer Distribution by RFM Segment',
             color='RFM_Segment',
             color_discrete_map=color_map)
fig.show()

Further classification of customers into business-relevant segments: (VIP/Loyal, Potential Loyal, At Risk, Can't Lose, Lost)

In [19]:
rfm['RFM_Customer_Segment'] = ''

rfm.loc[rfm['RFM_Score']>=9, 'RFM_Customer_Segment'] = 'VIP/Loyal'
rfm.loc[(rfm['RFM_Score']>=6) & (rfm['RFM_Score']<9), 'RFM_Customer_Segment'] = 'Potential Loyal'
rfm.loc[(rfm['RFM_Score']>=5) & (rfm['RFM_Score']<6), 'RFM_Customer_Segment'] = 'At Risk'
rfm.loc[(rfm['RFM_Score']>=4) & (rfm['RFM_Score']<5), 'RFM_Customer_Segment'] = 'Can\'t Lose'
rfm.loc[(rfm['RFM_Score']<4), 'RFM_Customer_Segment'] = 'Lost'
customer_segment_counts = rfm['RFM_Customer_Segment'].value_counts().sort_index()
customer_segment_counts

RFM_Customer_Segment
At Risk             517
Can't Lose          391
Lost                392
Potential Loyal    1382
VIP/Loyal          1690
Name: count, dtype: int64

In [20]:
segment_product_counts = rfm.groupby(['RFM_Segment', 'RFM_Customer_Segment']).size().reset_index(name='Count')
segment_product_counts = segment_product_counts.sort_values(by='Count', ascending=False)
segment_product_counts

,RFM_Segment,RFM_Customer_Segment,Count
0,High-Value,VIP/Loyal,1690
4,Mid-Value,Potential Loyal,1382
3,Mid-Value,At Risk,517
2,Low-Value,Lost,392
1,Low-Value,Can't Lose,391


Visualize the distribution of customers across RFM segments

In [21]:
fig_treemap_segment_product = px.treemap(segment_product_counts,
                                         path=['RFM_Segment', 'RFM_Customer_Segment'],
                                         values='Count',
                                         color = 'RFM_Segment',
                                         color_discrete_sequence=list(color_map.values()),
                                         title='RFM Customer Segments by Value')
fig_treemap_segment_product.show() 

##### 9. VIP Segment Analysis

In [22]:
vip_segment = rfm[rfm['RFM_Customer_Segment'] == 'VIP/Loyal']
vip_segment.head()

,Recency,Frequency,Monetary,R,F,M,RFM_Score,RFM_Segment,RFM_Customer_Segment
CustomerID,,,,,,,,,
12347.0,2,182,4310.00,4,4,4,12,High-Value,VIP/Loyal
12349.0,19,73,1757.55,3,3,4,10,High-Value,VIP/Loyal
12352.0,36,95,1545.41,3,3,3,9,High-Value,VIP/Loyal
12356.0,23,59,2811.43,3,3,4,10,High-Value,VIP/Loyal
12357.0,33,131,6207.67,3,4,4,11,High-Value,VIP/Loyal


In [23]:
fig = go.Figure()
fig.add_trace(go.Box(y=vip_segment['Recency'],name='Recency'))
fig.add_trace(go.Box(y=vip_segment['Frequency'], name='Frequency'))
fig.add_trace(go.Box(y=vip_segment['Monetary'], name='Monetary'))

In [24]:
correlation_matrix = vip_segment[['R', 'F','M']].corr()

In [25]:
correlation_matrix

,R,F,M
R,1.000000,-0.089849,-0.096407
F,-0.089849,1.000000,0.331813
M,-0.096407,0.331813,1.000000


In [26]:
fig_heatmap = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    colorscale='RdBu',
    colorbar=dict(title='Correlation')))
fig_heatmap.update_layout(title='Correlation Matrix of RFM Values within VIP Segment',
                              annotations=[
                                dict(
                                    x=x,
                                    y=y,
                                    text=str(correlation_matrix.loc[y, x].round(2)),
                                    showarrow=False,
                                    font=dict(color='black'))
                                for y in correlation_matrix.index
                                for x in correlation_matrix.columns
                            ])
fig_heatmap.show()

In [27]:
customer_segment_order = ['Lost', 'Can\'t Lose', 'At Risk', 'Potential Loyal', 'VIP/Loyal']
customer_segment_counts = customer_segment_counts.reindex(customer_segment_order)
customer_segment_counts

RFM_Customer_Segment
Lost                392
Can't Lose          391
At Risk             517
Potential Loyal    1382
VIP/Loyal          1690
Name: count, dtype: int64

##### 10. Segment Comparison

In [39]:
sequential_colors = ['#d62728', '#ff7f0e', '#ffeb3b', '#2ca02c', '#1f77b4']
fig = go.Figure(data=[go.Bar(x=customer_segment_counts.index,
                             y=customer_segment_counts.values,
                             marker=dict(color=sequential_colors))])

fig.update_traces(marker_color=[sequential_colors[i] 
                                 for i, segment_label in enumerate(customer_segment_counts.index)],
                                 marker_line_color='black',
                                 marker_line_width=1.5,opacity=0.6)

fig.update_layout(title='RFM Segments Distribution',
                  xaxis_title='RFM Segments',
                  yaxis_title='Number of Customers',
                  showlegend=False)
fig.show()

In [29]:
segment_scores = rfm.groupby('RFM_Customer_Segment')[['R', 'F', 'M']].mean().reset_index()

In [36]:
segment_scores['RFM_Customer_Segment'] = pd.Categorical(segment_scores['RFM_Customer_Segment'], 
                                                        categories=customer_segment_order, 
                                                        ordered=True)
segment_scores = segment_scores.sort_values('RFM_Customer_Segment')
segment_scores

,RFM_Customer_Segment,R,F,M
2,Lost,1.000000,1.000000,1.000000
1,Can't Lose,1.462916,1.273657,1.263427
0,At Risk,1.735010,1.624758,1.640232
3,Potential Loyal,2.444284,2.253256,2.301737
4,VIP/Loyal,3.404142,3.568639,3.559172


In [38]:
pastel_colors = px.colors.qualitative.Pastel
fig = go.Figure()

fig.add_trace(go.Bar(
    x=segment_scores['RFM_Customer_Segment'],
    y=segment_scores['R'],
    name='Recency',
    marker_color=pastel_colors[6]
))

fig.add_trace(go.Bar(
    x=segment_scores['RFM_Customer_Segment'],
    y=segment_scores['F'],
    name='Frequency',
    marker_color=pastel_colors[5]
))

fig.add_trace(go.Bar(
    x=segment_scores['RFM_Customer_Segment'],
    y=segment_scores['M'],
    name='Monetary',
    marker_color=pastel_colors[4]
))

fig.update_layout(
    title='Comparison of RFM Segments based on Recency, Frequency, and Monetary Scores',
    xaxis_title='RFM Segments',
    yaxis_title='Average Score',
    barmode='group',
    showlegend=True,
)

fig.show()

##### Key Insights

- **Distinct Customer Segments Identified:**  
  The RFM analysis successfully segmented customers into Low-Value, Mid-Value, and High-Value groups, as well as business-relevant segments such as Lost, Can't Lose, At Risk, Potential Loyal and VIP/Loyal.

- **Customer Distribution:**  
  The majority of customers fall into the Mid-Value segment, particularly the Potential Loyal segment, suggesting opportunities for targeted marketing to move them into the High-Value segment.


##### Recommendations:
  - Focus retention efforts on At Risk and Can't Lose segments to prevent churn.
  - Reward VIP/Loyal customers with exclusive offers.
  - Develop re-engagement campaigns for Lost customers.
  - Encourage Potential Loyal customers to increase their frequency and monetary scores.